In [2]:
import pandas as pd
import numpy as np
from collections import Counter

In [4]:
pip install openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.9/250.9 kB 6.4 MB/s eta 0:00:000:00:01
Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd

# Load the Excel file
excel_file = 'Prev_results/static_Final_Lexicon_4_LSVM.xlsx'  # Replace with your Excel file path

# Read the two sheets
df_severe = pd.read_excel(excel_file, sheet_name='Severe Lexicons')  # Replace with your sheet name if different
df_nonsevere = pd.read_excel(excel_file, sheet_name='NonSevere Lexicons')  # Replace with your sheet name if different

# Drop the 'Ratio' column if it exists
if 'Ratio' in df_severe.columns:
    df_severe = df_severe.drop(columns=['Ratio'])
if 'Ratio' in df_nonsevere.columns:
    df_nonsevere = df_nonsevere.drop(columns=['Ratio'])

# Convert the DataFrames to text files
df_severe.to_csv('severe_threshold_LSVM.txt', sep='\t', index=False)
df_nonsevere.to_csv('nonsevere_threshold_LSVM.txt', sep='\t', index=False)

print("Excel sheets have been converted to 'severe_lsvm.txt' and 'nonsevere_lsvm.txt' without the 'Ratio' column")


Excel sheets have been converted to 'severe_lsvm.txt' and 'nonsevere_lsvm.txt' without the 'Ratio' column


In [ ]:
# # Load positive and negative words from Bing Liu Lexicon
# with open('positive-words.txt', 'r') as f:
#     words = set(f.read().splitlines())

# with open('negative-words.txt', 'r') as f:
#     negative_words = set(f.read().splitlines())

In [ ]:
# bugs_eclipse = pd.read_csv("bugs_eclipse.csv")


# bugs_eclipse['Type'] = np.where(bugs_eclipse['Severity'] == 'enhancement', "enhancement", "defect")


# # Dropped rows with severity level '--'
# bugs_eclipse = bugs_eclipse[bugs_eclipse["Severity"].str.contains("--")==False].reset_index()

# #Dropped rows with Type "Enhancement" and "Task" because they are not a bug but a new feature
# indexSevere = bugs_eclipse[(bugs_eclipse['Type'] == 'enhancement') & (bugs_eclipse['Type'] == 'enhancement') ].index
# bugs_eclipse.drop(indexSevere , inplace=True)

# indexSevere = bugs_eclipse[(bugs_eclipse['Type'] == 'task') & (bugs_eclipse['Type'] == 'task') ].index
# bugs_eclipse.drop(indexSevere , inplace=True)


# #Catagorise the severity level into a Severe and Non Severe to make it a binary problem
# bugs_eclipse.loc[bugs_eclipse["Severity"] == "blocker", "Severity"] = 'Severe'
# bugs_eclipse.loc[bugs_eclipse["Severity"] == "critical", "Severity"] = 'Severe'
# bugs_eclipse.loc[bugs_eclipse["Severity"] == "major", "Severity"] = 'Severe'
# bugs_eclipse.loc[bugs_eclipse["Severity"] == "S1", "Severity"] = 'Severe'
# bugs_eclipse.loc[bugs_eclipse["Severity"] == "S2", "Severity"] = 'Severe'
# bugs_eclipse.loc[bugs_eclipse["Severity"] == "S3", "Severity"] = 'NonSevere'
# bugs_eclipse.loc[bugs_eclipse["Severity"] == "normal", "Severity"] = 'NonSevere'
# bugs_eclipse.loc[bugs_eclipse["Severity"] == "minor", "Severity"] = 'NonSevere'
# bugs_eclipse.loc[bugs_eclipse["Severity"] == "trivial", "Severity"] = 'NonSevere'
# bugs_eclipse.loc[bugs_eclipse["Severity"] == "S4", "Severity"] = 'NonSevere'

# df = bugs_eclipse

In [46]:
bugs_df = pd.read_csv("bugs_firefox.csv")
# Dropped rows with severity level '--'
bugs_df = bugs_df[bugs_df["Severity"].str.contains("--")==False].reset_index()

#Dropped rows with Type "Enhancement" and "Task" because they are not a bug but a new feature
indexSevere = bugs_df[ (bugs_df['Type'] == 'enhancement') & (bugs_df['Type'] == 'enhancement') ].index
bugs_df.drop(indexSevere , inplace=True)

indexSevere = bugs_df[ (bugs_df['Type'] == 'task') & (bugs_df['Type'] == 'task') ].index
bugs_df.drop(indexSevere , inplace=True)


#Catagorise the severity level into a Severe and Non Severe to make it a binary problem
bugs_df.loc[bugs_df["Severity"] == "blocker", "Severity"] = 'Severe'
bugs_df.loc[bugs_df["Severity"] == "critical", "Severity"] = 'Severe'
bugs_df.loc[bugs_df["Severity"] == "major", "Severity"] = 'Severe'
bugs_df.loc[bugs_df["Severity"] == "S1", "Severity"] = 'Severe'
bugs_df.loc[bugs_df["Severity"] == "S2", "Severity"] = 'Severe'
bugs_df.loc[bugs_df["Severity"] == "S3", "Severity"] = 'NonSevere'
bugs_df.loc[bugs_df["Severity"] == "normal", "Severity"] = 'NonSevere'
bugs_df.loc[bugs_df["Severity"] == "minor", "Severity"] = 'NonSevere'
bugs_df.loc[bugs_df["Severity"] == "trivial", "Severity"] = 'NonSevere'
bugs_df.loc[bugs_df["Severity"] == "S4", "Severity"] = 'NonSevere'

df = bugs_df

In [ ]:
def check_word_intersection(txt_file, df, Summary, severity_column):
    # Read the words from the text file
    with open(txt_file, 'r') as file:
        words = file.read().splitlines()
    
    # Initialize a Counter to keep track of word intersections
    word_counter = Counter()
    
    # Dictionaries to keep track of words in severe and non-severe categories
    severe_words = Counter()
    non_severe_words = Counter()
    
    # Function to find intersecting words and update the counters
    def find_intersections(text, severity):
        text_words = set(text.split())
        intersections = text_words.intersection(words)
        for word in intersections:
            word_counter[word] += 1
            if severity == 'Severe':
                severe_words[word] += 1
            elif severity == 'NonSevere':
                non_severe_words[word] += 1
        return ', '.join(intersections) if intersections else None
    
    # Check for intersections in the DataFrame column
    df['intersections'] = df.apply(lambda row: find_intersections(row[Summary], row[severity_column]), axis=1)
    
    # Words that appear in both categories
    both_categories = {word for word in words if severe_words[word] > 0 and non_severe_words[word] > 0}
    
    # Words that appear only in severe category
    only_severe = {word for word in words if severe_words[word] > 0 and non_severe_words[word] == 0}
    
    # Words that appear only in non-severe category
    only_non_severe = {word for word in words if non_severe_words[word] > 0 and severe_words[word] == 0}
    
    # Return the DataFrame with the intersections column and the word counters
    return df, word_counter, both_categories, only_severe, only_non_severe

# call function:

# Assuming df is already defined and has columns named 'Summary' and 'Severity'
result_df, word_counts, both_categories, only_severe, only_non_severe = check_word_intersection('nonsevere_threshold_LSVM.txt', df, 'Summary', 'Severity')
filtered_df = result_df[result_df['intersections'].notna()]

# Print the filtered DataFrame
# print(filtered_df['intersections'])
# print(set(filtered_df['intersections']))

# Print the word counts
print(word_counts)

# Print the categorized words
# print("Words in both categories:", both_categories)
print("Words only in severe category:", only_severe)
print("Words only in non-severe category:", only_non_severe)


Counter({'tab': 483, 'page': 353, 'bar': 322, 'window': 314, 'menu': 300, 'new': 281, 'button': 250, 'search': 181, 'displayed': 171, 'bug': 163, 'using': 161, 'browser': 157, 'text': 150, 'single': 148, 'tracking': 146, 'file': 141, 'icon': 135, 'mode': 134, 'panel': 131, 'toolbar': 128, 'default': 127, 'firefox': 125, 'dialog': 123, 'download': 106, 'opened': 102, 'link': 102, 'screen': 98, 'address': 98, 'instead': 97, 'user': 96, 'show': 95, 'focus': 95, 'bookmark': 92, 'set': 92, 'error': 92, 'opening': 91, 'option': 89, 'content': 89, 'wrong': 88, 'one': 86, 'list': 86, 'context': 86, 'click': 84, 'keyboard': 82, 'update': 80, 'history': 80, 'use': 79, 'do': 78, 'correctly': 74, 'mouse': 73, 'clicking': 72, 'timed': 71, 'test': 70, 'display': 70, 'theme': 69, 'session': 68, 'time': 66, 'even': 65, 'right': 65, 'multiple': 65, 'different': 64, 'close': 64, 'selected': 64, 'application': 63, 'size': 63, 'site': 62, 'first': 62, 'scroll': 61, 'folder': 59, 'top': 59, 'without': 59, 

In [48]:
def check_word_intersection(txt_file, df, column_name, severity_column):
    # Read the words from the text file
    with open(txt_file, 'r') as file:
        words = file.read().splitlines()
    
    # Initialize a Counter to keep track of word intersections
    word_counter = Counter()
    
    # Dictionaries to keep track of words in severe and non-severe categories
    severe_words = Counter()
    non_severe_words = Counter()
    
    # Function to find intersecting words and update the counters
    def find_intersections(text, severity):
        text_words = set(text.split())
        intersections = text_words.intersection(words)
        for word in intersections:
            word_counter[word] += 1
            if severity == 'Severe':
                severe_words[word] += 1
            elif severity == 'NonSevere':
                non_severe_words[word] += 1
        return ', '.join(intersections) if intersections else None
    
    # Check for intersections in the DataFrame column
    df['intersections'] = df.apply(lambda row: find_intersections(row[column_name], row[severity_column]), axis=1)
    
    # Words that appear in both categories
    both_categories = {word for word in words if severe_words[word] > 0 and non_severe_words[word] > 0}
    
    # Words that appear only in severe category
    only_severe = {word for word in words if severe_words[word] > 0 and non_severe_words[word] == 0}
    
    # Words that appear only in non-severe category
    only_non_severe = {word for word in words if non_severe_words[word] > 0 and severe_words[word] == 0}
    
    # Return the DataFrame with the intersections column and the word counters
    return df, word_counter, both_categories, only_severe, only_non_severe

# Example usage:

# Assuming df is already defined and has columns named 'Summary' and 'Severity'
result_df, word_counts, both_categories, only_severe, only_non_severe = check_word_intersection('severe_threshold_LSVM.txt', df, 'Summary', 'Severity')
filtered_df = result_df[result_df['intersections'].notna()]

# Print the filtered DataFrame
# print(filtered_df)
# print(set(filtered_df['intersections']))

# Print the word counts
print(word_counts)

# Print the categorized words
# print("Words in both categories:", both_categories)
print("Words only in severe category:", only_severe)
print("Words only in non-severe category:", only_non_severe)


Counter({'in': 1518, 'not': 1010, 'window': 314, 'menu': 300, 'new': 281, 'button': 250, 'open': 209, 'displayed': 171, 'bug': 163, 'using': 161, 'single': 148, 'tracking': 146, 'work': 137, 'panel': 131, 'toolbar': 128, 'download': 106, 'opened': 102, 'screen': 98, 'user': 96, 'error': 92, 'opening': 91, 'option': 89, 'content': 89, 'one': 86, 'context': 86, 'keyboard': 82, 'use': 79, 'fails': 74, 'clicking': 72, 'back': 71, 'working': 70, 'display': 70, 'session': 68, 'time': 66, 'multiple': 65, 'selected': 64, 'application': 63, 'load': 63, 'size': 63, 'cannot': 62, 'first': 62, 'loading': 59, 'top': 59, 'without': 59, 'via': 58, 'missing': 57, 'appears': 57, 'get': 56, 'closed': 56, 'sometimes': 56, 'field': 56, 'longer': 54, 'sync': 54, 'private': 54, 'saved': 50, 'properly': 49, 'still': 49, 'data': 49, 'save': 49, 'appear': 48, 'last': 47, 'always': 46, 'closing': 46, 'incorrect': 45, 'enabled': 45, 'line': 45, 'engine': 45, 'current': 44, 'message': 44, 'previous': 44, 'system'

In [49]:
# import pandas as pd
# from collections import Counter

# def check_word_intersection(txt_file, df, column_name, severity_column):
#     # Read the words from the text file
#     with open(txt_file, 'r') as file:
#         words = file.read().splitlines()
    
#     # Initialize a Counter to keep track of word intersections
#     word_counter = Counter()
    
#     # Dictionaries to keep track of words in severe and non-severe categories
#     severe_words = Counter()
#     non_severe_words = Counter()
    
#     # Function to find intersecting words and update the counters
#     def find_intersections(text, severity):
#         text_words = set(text.split())
#         intersections = text_words.intersection(words)
#         for word in intersections:
#             word_counter[word] += 1
#             if severity == 'Severe':
#                 severe_words[word] += 1
#             elif severity == 'Non-Severe':
#                 non_severe_words[word] += 1
#         return ', '.join(intersections) if intersections else None
    
#     # Check for intersections in the DataFrame column
#     df['intersections'] = df.apply(lambda row: find_intersections(row[column_name], row[severity_column]), axis=1)
    
#     # Debug prints to check the counts
#     print("Severe words count:", severe_words)
#     print("Non-severe words count:", non_severe_words)
    
#     # Words that appear in both categories
#     both_categories = {word for word in words if severe_words[word] > 0 and non_severe_words[word] > 0}
    
#     # Words that appear only in severe category
#     only_severe = {word for word in words if severe_words[word] > 0 and non_severe_words[word] == 0}
    
#     # Words that appear only in non-severe category
#     only_non_severe = {word for word in words if non_severe_words[word] > 0 and severe_words[word] == 0}
    
#     # Return the DataFrame with the intersections column and the word counters
#     return df, word_counter, both_categories, only_severe, only_non_severe

# # Example usage:

# # Assuming df is already defined and has columns named 'Summary' and 'Severity'
# result_df, word_counts, both_categories, only_severe, only_non_severe = check_word_intersection('positive-words.txt', df, 'Summary', 'Severity')
# filtered_df = result_df[result_df['intersections'].notna()]

# # Print the filtered DataFrame
# print(filtered_df)

# # Print the word counts
# print(word_counts)

# # Print the categorized words
# print("Words in both categories:", both_categories)
# print("Words only in severe category:", only_severe)
# print("Words only in non-severe category:", only_non_severe)


In [50]:
# def check_word_intersection(txt_file, df, column_name, severity_column):
#     # Read the words from the text file
#     with open(txt_file, 'r') as file:
#         words = file.read().splitlines()
    
#     # Initialize a Counter to keep track of word intersections
#     word_counter = Counter()
    
#     # Dictionaries to keep track of words in severe and non-severe categories
#     severe_words = Counter()
#     non_severe_words = Counter()
    
#     # Function to find intersecting words and update the counters
#     def find_intersections(text, severity):
#         text_words = set(text.split())
#         intersections = text_words.intersection(words)
#         for word in intersections:
#             word_counter[word] += 1
#             if severity == 'Severe':
#                 severe_words[word] += 1
#             elif severity == 'Non-Severe':
#                 non_severe_words[word] += 1
#         return ', '.join(intersections) if intersections else None
    
#     # Check for intersections in the DataFrame column
#     df['intersections'] = df.apply(lambda row: find_intersections(row[column_name], row[severity_column]), axis=1)
    
#     # Words that appear in both categories
#     both_categories = {word for word in words if severe_words[word] > 0 and non_severe_words[word] > 0}
    
#     # Words that appear only in severe category
#     only_severe = {word for word in words if severe_words[word] > 0 and non_severe_words[word] == 0}
    
#     # Words that appear only in non-severe category
#     only_non_severe = {word for word in words if non_severe_words[word] > 0 and severe_words[word] == 0}
    
#     # Return the DataFrame with the intersections column and the word counters
#     return df, word_counter, both_categories, only_severe, only_non_severe

# # Example usage:

# # Assuming df is already defined and has columns named 'Summary' and 'Severity'
# result_df, word_counts, both_categories, only_severe, only_non_severe = check_word_intersection('positive-words.txt', df, 'Summary', 'Severity')
# filtered_df = result_df[result_df['intersections'].notna()]

# # Print the filtered DataFrame
# print(filtered_df)

# # Print the word counts
# print(word_counts)

# # Print the categorized words
# print("Words in both categories:", both_categories)
# print("Words only in severe category:", only_severe)
# print("Words only in non-severe category:", only_non_severe)


In [51]:
# def check_word_intersection(txt_file, df, column_name, severity_column):
#     # Read the words from the text file
#     with open(txt_file, 'r') as file:
#         words = file.read().splitlines()
    
#     # Initialize a Counter to keep track of word intersections
#     word_counter = Counter()
    
#     # Dictionaries to keep track of words in severe and non-severe categories
#     severe_words = Counter()
#     non_severe_words = Counter()
    
#     # Function to find intersecting words and update the counters
#     def find_intersections(text, severity):
#         text_words = set(text.split())
#         intersections = text_words.intersection(words)
#         for word in intersections:
#             word_counter[word] += 1
#             if severity == 'Severe':
#                 severe_words[word] += 1
#             elif severity == 'Non-Severe':
#                 non_severe_words[word] += 1
#         return ', '.join(intersections) if intersections else None
    
#     # Check for intersections in the DataFrame column
#     df['intersections'] = df.apply(lambda row: find_intersections(row[column_name], row[severity_column]), axis=1)
    
#     # Words that appear in both categories
#     both_categories = {word for word in words if severe_words[word] > 0 and non_severe_words[word] > 0}
    
#     # Words that appear only in severe category
#     only_severe = {word for word in words if severe_words[word] > 0 and non_severe_words[word] == 0}
    
#     # Words that appear only in non-severe category
#     only_non_severe = {word for word in words if non_severe_words[word] > 0 and severe_words[word] == 0}
    
#     # Return the DataFrame with the intersections column and the word counters
#     return df, word_counter, both_categories, only_severe, only_non_severe

# # Example usage:

# # Assuming df is already defined and has columns named 'Summary' and 'Severity'
# result_df, word_counts, both_categories, only_severe, only_non_severe = check_word_intersection('negative-words.txt', df, 'Summary', 'Severity')
# filtered_df = result_df[result_df['intersections'].notna()]

# # Print the filtered DataFrame
# print(filtered_df)

# # Print the word counts
# print(word_counts)

# # Print the categorized words
# print("Words in both categories:", both_categories)
# print("Words only in severe category:", only_severe)
# print("Words only in non-severe category:", only_non_severe)


In [52]:
# import pandas as pd
# from collections import Counter

# def check_word_intersection(txt_file, df, column_name):
#     # Read the words from the text file
#     with open(txt_file, 'r') as file:
#         words = file.read().splitlines()
    
#     # Initialize a Counter to keep track of word intersections
#     word_counter = Counter()
    
#     # Function to find intersecting words and update the counter
#     def find_intersections(text):
#         text_words = set(text.split())
#         intersections = text_words.intersection(words)
#         for word in intersections:
#             word_counter[word] += 1
#         return ', '.join(intersections) if intersections else None
    
#     # Check for intersections in the DataFrame column
#     df['intersections'] = df[column_name].apply(find_intersections)
    
#     # Return the DataFrame with the intersections column and the word counter
#     return df, word_counter

# # Example usage:

# # Assuming df is already defined and has a column named 'Summary'
# result_df, word_counts = check_word_intersection('positive-words.txt', df, 'Summary')
# filtered_df = result_df[result_df['intersections'].notna()]

# # Print the filtered DataFrame
# print(filtered_df)

# # Print the word counts
# print(word_counts)

In [53]:
# bugs_df['Processed_Summary'] = bugs_df['Summary'].apply(lambda x: helper.nlpsteps(x))

In [54]:

# def check_word_intersection(txt_file, df, column_name):
#     # Read the words from the text file
#     with open(txt_file, 'r') as file:
#         words = file.read().splitlines()
    
#     # Function to find intersecting words
#     def find_intersections(text):
#         text_words = set(text.split())
#         intersections = text_words.intersection(words)
#         return ', '.join(intersections) if intersections else None
    
#     # Check for intersections in the DataFrame column
#     df['intersections'] = df[column_name].apply(find_intersections)
    
#     # Return the DataFrame with the intersections column
#     return df

# # Example usage:

# # Assuming df is already defined and has a column named 'Summary'
# result_df = check_word_intersection('positive-words.txt', df, 'Summary')
# filtered_df = result_df[result_df['intersections'].notna()]

# # Print the filtered DataFrame
# print(set(filtered_df['intersections']))
# # print(result_df)


In [55]:
# import pandas as pd

# def check_word_intersection(txt_file, df, column_name):
#     # Read the words from the text file
#     with open(txt_file, 'r') as file:
#         words = file.read().splitlines()
    
#     # Function to find intersecting words
#     def find_intersections(text):
#         text_words = set(text.split())
#         intersections = text_words.intersection(words)
#         return ', '.join(intersections) if intersections else None
    
#     # Check for intersections in the DataFrame column
#     df['intersections'] = df[column_name].apply(find_intersections)
    
#     # Return the DataFrame with the intersections column
#     return df

# # Example usage:

# # Assuming df is already defined and has a column named 'Summary'
# result_df = check_word_intersection('negative-words.txt', df, 'Summary')
# filtered_df = result_df[result_df['intersections'].notna()]

# # Print the filtered DataFrame
# print(set(filtered_df['intersections']))
# # print(result_df)
